In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob
import pickle
import helper_functions_06b as hf

In [2]:
DATE = ['20230306', '20230307', '20230314', '20230317', '20230317']
SESSION = ['SE_Lek1', 'SM_Lek1', 'SE_Lek1', 'SM_Lek1', 'SM_Lek1']
DRONE = ['P3D6', 'P1D2', 'P3D6', 'P2D3', 'P2D4']
VIDEOS = ['0663', '0796', '0978', '0410', '0035']

tracking_directory = '/Volumes/EAS_shared/blackbuck/working/processed/Field_Recording_2023/SpatialRegistration'

In [3]:
# Thresholds for translation, rotation, and scale
thresholds = {
    'T_translation': 30,  # e.g., 30 pixels
    'T_rotation': 20,     # e.g., 20 degrees
    'T_scale': 0.1         # e.g., 10% change
}

In [4]:
for date,session,drone,video in zip(DATE,SESSION,DRONE,VIDEOS):
    internal_path = tracking_directory + '/' + date + '/' + session + '/' + drone
    files = glob.glob(internal_path + '/' + date + '_' + session + '_' + drone + '_DJI_' + video + '_YOLO_tracked_*.csv')
    
    file = files[0]
    print(date, session, drone, os.path.basename(file))

    df1 = pd.read_csv(file)
    df1['x'] = df1['bb_left'] + df1['bb_width']/2
    df1['y'] = df1['bb_top'] + df1['bb_height']/2
    df1['idx'] = df1['track_id']

    df1 = df1.loc[:,['frame', 'x', 'y', 'idx']]

    # Step 1: Identify anchor frames
    anchor_frames1 = hf.identify_anchor_frames(df1, thresholds)

    # Step 2: Find the best anchor for each frame
    df1 = hf.find_best_anchor_for_each_frame_parallel(df1, anchor_frames1)

    # Step 3: Apply homography using the best anchor for each frame
    df1, homography_matrices1, rms_errors = hf.apply_homography_to_best_anchor(df1)
    
    #######################################################
    #######################################################
    #######################################################
    
    file = files[1]
    print(date, session, drone, os.path.basename(file))

    df2 = pd.read_csv(file)
    df2['x'] = df2['bb_left'] + df2['bb_width']/2
    df2['y'] = df2['bb_top'] + df2['bb_height']/2
    df2['idx'] = df2['track_id']

    df2 = df2.loc[:,['frame', 'x', 'y', 'idx']]

    # Step 1: Identify anchor frames
    anchor_frames2 = hf.identify_anchor_frames(df2, thresholds)

    # Step 2: Find the best anchor for each frame
    df2 = hf.find_best_anchor_for_each_frame_parallel(df2, anchor_frames2)

    # Step 3: Apply homography using the best anchor for each frame
    df2, homography_matrices2, rms_errors = hf.apply_homography_to_best_anchor(df2)

    df = pd.concat((df1, df2), axis=0)
    homography_matrices = {**homography_matrices1, **homography_matrices2}
    
    anchor_filename = os.path.basename(file).replace('_YOLO_tracked_end.csv', '_Anchored.csv')
    pickle_filename = os.path.basename(file).replace('_YOLO_tracked_end.csv', '_homographies.pkl')
    df.to_csv(os.path.join(os.path.dirname(file), anchor_filename), index=False, mode='w')
    with open(os.path.join(os.path.dirname(file), pickle_filename), 'wb') as f:
        pickle.dump(homography_matrices, f)

20230306 SE_Lek1 P3D6 20230306_SE_Lek1_P3D6_DJI_0663_YOLO_tracked_start.csv
Initial identification of anchors takes  7.380695104598999
Finding the best anchors took 157.00755095481873
Applying homography to the best anchors takes  39.20904207229614
20230306 SE_Lek1 P3D6 20230306_SE_Lek1_P3D6_DJI_0663_YOLO_tracked_end.csv
Initial identification of anchors takes  7.068833827972412
Finding the best anchors took 244.3058660030365
Applying homography to the best anchors takes  23.26427412033081
20230307 SM_Lek1 P1D2 20230307_SM_Lek1_P1D2_DJI_0796_YOLO_tracked_start.csv
Initial identification of anchors takes  11.370456218719482
Finding the best anchors took 291.5494191646576
Applying homography to the best anchors takes  84.98406624794006
20230307 SM_Lek1 P1D2 20230307_SM_Lek1_P1D2_DJI_0796_YOLO_tracked_end.csv
Initial identification of anchors takes  5.137281179428101
Finding the best anchors took 182.5832018852234
Applying homography to the best anchors takes  22.36190629005432
20230314 S